In [1]:
Pkg.add("POMDPs")
using POMDPs

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of POMDPs
INFO: Use `Pkg.update()` to get the latest versions of your packages


In [2]:
POMDPs.add("POMDPToolbox")
using POMDPToolbox # for DiscreteBelief, HistoryRecorder, etc
typealias Categorical DiscreteBelief # just a vector of probabilities

Package already installed


INFO: Cloning POMDPToolbox from https://github.com/JuliaPOMDP/POMDPToolbox.jl


POMDPToolbox.DiscreteBelief

In [3]:
importall POMDPs
include("helpers.jl");

LoadError: LoadError: could not open file /home/juser/NeedlePOMDP/examples/helpers.jl
while loading In[3], in expression starting on line 2

In [4]:
plot_chain(4, fill=Dict(4=>"green"))

LoadError: LoadError: UndefVarError: plot_chain not defined
while loading In[4], in expression starting on line 1

In [5]:
type ChainMDP <: MDP{Int, Bool} # left is false, right is true
    len::Int
    p_success::Float64
    discount::Float64
end
discount(mdp::ChainMDP) = mdp.discount

discount (generic function with 2 methods)

In [6]:
create_transition_distribution(mdp::ChainMDP) = Categorical(mdp.len)

create_transition_distribution (generic function with 2 methods)

In [7]:
# the transition function returns the distribution of sp given that action a is taken in state s
function transition(mdp::ChainMDP, s::Int, a::Bool, d::Categorical=create_transition_distribution(mdp))
    fill!(d, 0.0)
    if a == true
        success = min(s+1, mdp.len)
        failure = max(s-1, 1)
    else # false
        success = max(s-1, 1)
        failure = min(s+1, mdp.len)
    end
    d[success] = mdp.p_success
    d[failure] = 1.0 - mdp.p_success
    return d
end

transition (generic function with 4 methods)

In [8]:
function reward(mdp::ChainMDP, s::Int, a::Bool)
    if s == mdp.len
        return 1.0
    else
        return 0.0
    end 
end

reward (generic function with 3 methods)

In [9]:
mdp = ChainMDP(4, 0.7, 0.9)

ChainMDP(4,0.7,0.9)

In [10]:
transition(mdp, 1, true)

POMDPToolbox.DiscreteBelief([0.3,0.7,0.0,0.0])

In [11]:
reward(mdp, 1, true)

0.0

In [12]:
reward(mdp, 1, true)

0.0

In [13]:
rec = HistoryRecorder(max_steps=100);

In [14]:
policy = VectorPolicy(mdp, fill(true, 4))

POMDPToolbox.VectorPolicy{Int64,Bool}(ChainMDP(4,0.7,0.9),Bool[true,true,true,true])

In [15]:
simulate(rec, mdp, policy, 1)

2.5514399145469504

In [16]:
using MCTS
solver = MCTSSolver(enable_tree_vis=true)
mcts_pol = solve(solver, mdp);

In [17]:
simulate(rec, mdp, mcts_pol, 1)

6.011218487714211

In [18]:
TreeVisualizer(mcts_pol, 1)

MCTS.TreeVisualizer{MCTS.MCTSPolicy{Int64,Bool,Void}}(MCTS.MCTSPolicy{Int64,Bool,Void}(MCTS.MCTSSolver(100,10,1.0,MersenneTwister(UInt32[0x00000000],Base.dSFMT.DSFMT_state(Int32[-837992505,1073430652,653755050,1073432280,-881398583,1072864555,740330250,1073725942,99456662,1073545709  …  -131277659,1072883343,-426838800,1072810381,-908659251,-459045743,-359578457,570071932,382,0]),[1.70324,1.7048,1.16337,1.98485,1.81297,1.09254,1.26,1.92402,1.13971,1.77571  …  1.9399,1.48986,1.14969,1.99687,1.80173,1.96197,1.8768,1.35384,1.18129,1.11171],329),POMDPToolbox.RandomSolver(MersenneTwister(UInt32[0x00000000],Base.dSFMT.DSFMT_state(Int32[-837992505,1073430652,653755050,1073432280,-881398583,1072864555,740330250,1073725942,99456662,1073545709  …  -131277659,1072883343,-426838800,1072810381,-908659251,-459045743,-359578457,570071932,382,0]),[1.70324,1.7048,1.16337,1.98485,1.81297,1.09254,1.26,1.92402,1.13971,1.77571  …  1.9399,1.48986,1.14969,1.99687,1.80173,1.96197,1.8768,1.35384,1.18129,1.11171],329)),nothing,true),ChainMDP(4,0.7,0.9),POMDPToolbox.RandomPolicy(MersenneTwister(UInt32[0x00000000],Base.dSFMT.DSFMT_state(Int32[-837992505,1073430652,653755050,1073432280,-881398583,1072864555,740330250,1073725942,99456662,1073545709  …  -131277659,1072883343,-426838800,1072810381,-908659251,-459045743,-359578457,570071932,382,0]),[1.70324,1.7048,1.16337,1.98485,1.81297,1.09254,1.26,1.92402,1.13971,1.77571  …  1.9399,1.48986,1.14969,1.99687,1.80173,1.96197,1.8768,1.35384,1.18129,1.11171],329),ChainMDP(4,0.7,0.9),POMDPToolbox.VoidUpdater(),(true,false)),Dict(4=>MCTS.StateNode{Bool}(62717,MCTS.StateActionNode{Bool}[MCTS.StateActionNode{Bool}(true,62693,3.12679,Set(Any[4,3])),MCTS.StateActionNode{Bool}(false,24,1.75319,Set(Any[4,3]))]),2=>MCTS.StateNode{Bool}(9088,MCTS.StateActionNode{Bool}[MCTS.StateActionNode{Bool}(true,9067,0.933572,Set(Any[3,1])),MCTS.StateActionNode{Bool}(false,21,0.297582,Set(Any[3,1]))]),3=>MCTS.StateNode{Bool}(24717,MCTS.StateActionNode{Bool}[MCTS.StateActionNode{Bool}(true,24714,1.84685,Set(Any[4,2])),MCTS.StateActionNode{Bool}(false,3,0.0,Set(Any[2]))]),1=>MCTS.StateNode{Bool}(3447,MCTS.StateActionNode{Bool}[MCTS.StateActionNode{Bool}(true,3344,0.478667,Set(Any[2,1])),MCTS.StateActionNode{Bool}(false,103,0.236344,Set(Any[2,1]))]))),1)

In [19]:
POMDPs.add("DiscreteValueIteration")

Package already installed


INFO: Cloning DiscreteValueIteration from https://github.com/JuliaPOMDP/DiscreteValueIteration.jl


In [20]:
using DiscreteValueIteration

In [21]:
n_states(mdp::ChainMDP) = mdp.len
states(mdp::ChainMDP) = 1:mdp.len

states (generic function with 6 methods)

In [22]:
solver = ValueIterationSolver()
vi_policy = solve(solver, mdp)
simulate(rec, mdp, vi_policy, 1)

2.6416001153803537

In [23]:
@less create_policy(solver, mdp)

# The solver type 
type ValueIterationSolver <: Solver
    max_iterations::Int64 # max number of iterations 
    belres::Float64 # the Bellman Residual
end
# Default constructor
function ValueIterationSolver(;max_iterations::Int64=100, belres::Float64=1e-3)
    return ValueIterationSolver(max_iterations, belres)
end

# The policy type
type ValueIterationPolicy <: Policy
    qmat::Matrix{Float64} # Q matrix stroign Q(s,a) values
    util::Vector{Float64} # The value function V(s)
    policy::Vector{Int64} # Policy array, maps state index to action index
    action_map::Vector{Any} # Maps the action index to the concrete action type
    include_Q::Bool # Flag for including the Q-matrix
    mdp::Union{MDP,POMDP} # uses the model for indexing in the action function
    # constructor with an optinal initial value function argument
    function ValueIterationPolicy(mdp::Union{MDP,POMDP}; 
                                  utility::Vector{Float64}=Array(Float64,0),
                           

In [24]:
POMDPs.add("POMDPModels")
using POMDPModels

Package already installed


INFO: Cloning POMDPModels from https://github.com/JuliaPOMDP/POMDPModels.jl


In [25]:
bp = BabyPOMDP()
up = BabyBeliefUpdater(bp)

POMDPModels.BabyBeliefUpdater(POMDPModels.BabyPOMDP(-5.0,-10.0,0.1,0.8,0.1,0.9,[0.0]))

In [26]:
POMDPs.add("SARSOP") # might take a while to compile
using SARSOP

Package already installed


INFO: Cloning SARSOP from https://github.com/JuliaPOMDP/SARSOP.jl


In [27]:
solver = SARSOPSolver()

SARSOP.SARSOPSolver(Dict{AbstractString,Any}())

In [28]:
p = solve(solver, bp)

Generating a pomdpx file: model.pomdpx

Loading the model ...
  input file   : model.pomdpx
  loading time : 0.00s 

SARSOP initializing ...
  initialization time : 0.00s

-------------------------------------------------------------------------------
 Time   |#Trial |#Backup |LBound    |UBound    |Precision  |#Alphas |#Beliefs  
-------------------------------------------------------------------------------
 0       0       0        -47.3685   -16.0713   31.2971     2        1        
 0       5       51       -17.9323   -16.0492   1.88318     9        6        
 0       10      103      -16.3978   -16.2159   0.181953    2        8        
 0       14      151      -16.3124   -16.277    0.0353847   2        13       
 0       18      207      -16.3058   -16.2969   0.00887227  2        16       
 0       21      250      -16.3055   -16.3022   0.00329006  7        16       
 0       25      305      -16.3055   -16.3044   0.00111584  2        18       
 0       26      323      -16.3055 

SARSOP.POMDPPolicy("out.policy",POMDPXFiles.POMDPAlphas([-29.6749 -38.2512; -19.6749 -16.3055],[0,1]),POMDPModels.BabyPOMDP(-5.0,-10.0,0.1,0.8,0.1,0.9,[0.0]),Any[true,false])

In [29]:
simulate(rec, bp, p, up, BoolDistribution(0.0))

-23.781178142898746

In [30]:
for i in 1:length(rec.action_hist)
    @printf("%3.2f hungry belief => %s\n",
            rec.belief_hist[i].p,
            rec.action_hist[i] ? "feed":"don't feed")
end

0.00 hungry belief => don't feed
0.47 hungry belief => feed
0.00 hungry belief => don't feed
0.47 hungry belief => feed
0.00 hungry belief => don't feed
0.47 hungry belief => feed
0.00 hungry belief => don't feed
0.02 hungry belief => don't feed
0.03 hungry belief => don't feed
0.54 hungry belief => feed
0.00 hungry belief => don't feed
0.02 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.54 hungry belief => feed
0.00 hungry belief => don't feed
0.02 hungry belief => don't feed
0.03 hungry belief => don't feed
0.03 hungry belief => don't feed
0.54 hungry belief => feed
0.00 hungry belief => don't feed
0.02 hungry belief => don't feed
0.03 hungry belief => don't feed
0.54 hungry belief => feed
0.00 hungry belief => don't feed
0.02 hungry belief => don't feed
0.03 hungry belief 